In [1]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark import SparkContext

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/23 15:09:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Question 1 

In [3]:
spark.version

'4.1.1'

# Question 2 

In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2025-11.parquet')

In [5]:
df = df.repartition(4)

In [6]:
df.write.parquet('data/pq/yellow/2025/11/')

In [19]:
ls -lh data/pq/yellow/2025/11 | awk '{print substr($0, index($0,$9)), $6}'

total 205312 
15:10 _SUCCESS 0B
15:10 part-00000-a6b110a1-0309-42fe-be8e-91c3ddc07edf-c000.snappy.parquet 24M
15:10 part-00001-a6b110a1-0309-42fe-be8e-91c3ddc07edf-c000.snappy.parquet 24M
15:10 part-00002-a6b110a1-0309-42fe-be8e-91c3ddc07edf-c000.snappy.parquet 24M
15:10 part-00003-a6b110a1-0309-42fe-be8e-91c3ddc07edf-c000.snappy.parquet 24M


# Question 3 

In [20]:
from pyspark.sql import functions as F

In [21]:
df = spark.read.parquet('data/pq/yellow/2025/11/')

In [22]:
df.registerTempTable('yellow')

/opt/homebrew/Caskroom/miniforge/base/lib/python3.12/site-packages/pyspark/sql/classic/dataframe.py:178: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [25]:
spark.sql("""
    SELECT COUNT(*) total_trips
    FROM yellow
    WHERE TO_DATE(tpep_pickup_datetime) = '2025-11-15'
""").show()

+-----------+
|total_trips|
+-----------+
|     162604|
+-----------+



# Question 4

In [24]:
spark.sql("""
    SELECT 
        MAX((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600) as max_trip_hours
    FROM yellow
""").show()

+-----------------+
|   max_trip_hours|
+-----------------+
|90.64666666666666|
+-----------------+



# Question 5

Spark's Application UI runs on port 4040 by default.

# Question 6

In [26]:
df_zones = spark.read \
            .option("header", "true") \
            .csv('taxi_zone_lookup.csv')

In [27]:
df_zones.registerTempTable('zones')

In [28]:
spark.sql("""
    SELECT 
        z.Zone,
        COUNT(*) as pickup_count
    FROM yellow y
    JOIN zones z ON y.PULocationID = z.LocationID
    GROUP BY z.Zone
    ORDER BY pickup_count ASC
    LIMIT 10
""").show()

[Stage 16:========================================>               (10 + 4) / 14]

+--------------------+------------+
|                Zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
|       Arden Heights|           1|
|Eltingville/Annad...|           1|
|       Port Richmond|           3|
|   Rossville/Woodrow|           4|
| Green-Wood Cemetery|           4|
|       Rikers Island|           4|
|         Great Kills|           4|
|         Jamaica Bay|           5|
|         Westerleigh|          12|
+--------------------+------------+

